## This jupyter note book shows to export data from redcap and extract used data used to plot KM curve and connect to tableau through web data connector

In [9]:
import pandas as pd
import jupytab
import numpy as np
import pandas as pd
import csv
import requests
import sys
from datetime import datetime
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import warnings
warnings.filterwarnings("ignore")

In [10]:
# this part for connect to redcap to get data

data = {
    'token': '5384D3408C5BD6C3611DFFCA1D3B4E7F',
    'content': 'record',
    'action': 'export',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://redcap.wehi.edu.au/api/',data=data)
print('HTTP Status: ' + str(r.status_code))
data = r.text
extracted_data = pd.read_csv(StringIO(data), sep=",")

HTTP Status: 200


In [12]:
extracted_data
Patients = extracted_data[extracted_data['redcap_repeat_instrument'].isnull()]
Medications = extracted_data[extracted_data['redcap_repeat_instrument'] == 'medications']
Conditions = extracted_data[extracted_data['redcap_repeat_instrument'] == 'conditions']

In [13]:
Patients = Patients.rename(columns = {'id_patient':'Id',
                                     'deathdate_patient':'DEATHDATE',
                                     'birthdate_patient':'BIRTHDATE'})


Medications = Medications.rename(columns = {'id_patient':'PATIENT',
                                     'description_medication':'DESCRIPTION',
                                           'start_medication':'START',
                                           'reasondescription_medication':'REASONDESCRIPTION'})


Conditions = Conditions.rename(columns = {'id_patient':'PATIENT',
                                     'description_condition':'DESCRIPTION',
                                         'start_condition':'START'})

In [14]:
# the function is used to get the data including (time_to_event, status) can be pluged into kaplanmeier estimator
def get_standard_format_medication(Medications,Patient,Conditions,Condition,medication_set,survival_analysis_endtime):

    # get the all medication information for the selected condition patients
    Medications['DESCRIPTION'] = Medications['DESCRIPTION'].apply(lambda x: x.lower())
    medication_1 = medication_set[0].lower()
    medication_2 = medication_set[1].lower()

    date_format1 = "%Y-%m-%d"

    # change name all to lowercase

    # condition on the condition(cancer type) and medication
    selected_Conditions = Conditions.loc[Conditions['DESCRIPTION'] == Condition]
    selected_Medications =  Medications.loc[(Medications.DESCRIPTION == medication_1) | (Medications.DESCRIPTION == medication_2)]
    
    # join the medication and condition information
    selected_data = selected_Conditions.merge(selected_Medications, how='inner', left_on=['PATIENT', 'DESCRIPTION'], right_on=['PATIENT', 'REASONDESCRIPTION'],suffixes=['_condition','_medication'])

    # change to datetime type for time information
    selected_data['START_medication'] = selected_data['START_medication'].apply(lambda row : datetime.strptime(str(row)[:10],date_format1) if (type(row)==str) else 'not defined' )
    selected_data['START_condition'] = selected_data['START_condition'].apply(lambda row : datetime.strptime(str(row)[:10],date_format1) if (type(row)==str) else 'not defined' )
    
    # get the earliest_medication_starttime for the two medicine for each patient
    earliest_medication_starttime = selected_data.groupby(['PATIENT', 'DESCRIPTION_medication','START_condition'])['START_medication'].min().reset_index(name='earliest_medication_starttime')
   
    # join with patient.csv to get deathdate information
    selected_data = Patient.merge(earliest_medication_starttime, how='inner', left_on=['Id'], right_on=['PATIENT'])
    

    #get status for the event(death as 1, alive as 0)
    selected_data['Status'] = selected_data['DEATHDATE'].apply(lambda row : 1 if (type(row)==str) else 0 )
    selected_data['start_time'] = selected_data['earliest_medication_starttime']
    
    # data must have start_time
    selected_data = selected_data[selected_data['start_time'].notna()]
  

    #get end_time for events( if alive then use the survival_analysis_endtime else use the deathtime
    selected_data['end_time'] = selected_data['DEATHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],date_format1) if (type(row)==str) else survival_analysis_endtime)
    selected_data['Time'] = (selected_data['end_time'] - selected_data['start_time'])
    selected_data['BIRTHDATE'] = selected_data['BIRTHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],date_format1) if (type(row)==str) else 'not defined' )
    selected_data['Age_cancer'] = (selected_data['START_condition'] - selected_data['BIRTHDATE'])
    selected_data['Age_cancer'] = round(selected_data['Age_cancer'].dt.days/365)
    
    
    selected_data =selected_data.rename(columns = {'DESCRIPTION_medication':'Medication'})
    return selected_data

In [15]:
medication_set = ['100 ML Epirubicin Hydrochloride 2 MG/ML Injection','Paclitaxel 100 MG Injection']
Condition = 'Malignant neoplasm of breast (disorder)'
date_format1 = "%Y-%m-%d"
start_point = False
survival_analysis_endtime = datetime.strptime('2023-1-21',date_format1)
selected_data = get_standard_format_medication(Medications,Patients,Conditions,Condition,medication_set,survival_analysis_endtime)

In [16]:
standard_format = selected_data[['Id','Time','Status','Medication','Age_cancer']]

In [17]:
standard_format


,Id,Time,Status,Medication,Age_cancer
0,0dbd5054-38f2-ca4a-3222-1231810defa2,320 days,1,100 ml epirubicin hydrochloride 2 mg/ml injection,1.0
1,3fd52c98-62e8-019b-db8c-dfa56cae06eb,3693 days,0,100 ml epirubicin hydrochloride 2 mg/ml injection,65.0
2,7fe0a247-811b-fc51-4e15-ce39e06c2394,823 days,0,paclitaxel 100 mg injection,58.0
3,9a87803d-2a1b-db4c-979a-816bf219c458,745 days,1,paclitaxel 100 mg injection,44.0
4,9e9047d5-4f5e-dbc8-2dae-b2b43a6246ae,816 days,0,paclitaxel 100 mg injection,52.0
5,24a00ffd-375f-056a-7e02-d8ddc6ff2de4,1806 days,1,paclitaxel 100 mg injection,67.0
6,39df748d-1ead-461d-5ed5-54fa622396f1,3066 days,0,paclitaxel 100 mg injection,61.0
7,71d99b97-f1ff-579f-a626-619ad77fda13,3262 days,1,paclitaxel 100 mg injection,43.0
8,86abebe8-0e13-ebe4-99e1-e91918d8f5a7,48 days,1,paclitaxel 100 mg injection,12.0
9,483ae6d2-0ae6-e8ec-4eed-5ce2a4a64143,1249 days,1,paclitaxel 100 mg injection,58.0


In [18]:
# Note here the time is in type of datetime, we need to change to integer
standard_format['Time'] = standard_format['Time'].dt.days

In [19]:
if start_point==False:
    for medication_type in np.unique(standard_format.Medication):
        print(medication_type)
        new_row = {"Id":"Added","Time":0,"Status":0,"Medication": medication_type}
        standard_format = standard_format.append(new_row, ignore_index=True)
    start_point = True


100 ml epirubicin hydrochloride 2 mg/ml injection
paclitaxel 100 mg injection


In [20]:
standard_format

,Id,Time,Status,Medication,Age_cancer
0,0dbd5054-38f2-ca4a-3222-1231810defa2,320,1,100 ml epirubicin hydrochloride 2 mg/ml injection,1.0
1,3fd52c98-62e8-019b-db8c-dfa56cae06eb,3693,0,100 ml epirubicin hydrochloride 2 mg/ml injection,65.0
2,7fe0a247-811b-fc51-4e15-ce39e06c2394,823,0,paclitaxel 100 mg injection,58.0
3,9a87803d-2a1b-db4c-979a-816bf219c458,745,1,paclitaxel 100 mg injection,44.0
4,9e9047d5-4f5e-dbc8-2dae-b2b43a6246ae,816,0,paclitaxel 100 mg injection,52.0
5,24a00ffd-375f-056a-7e02-d8ddc6ff2de4,1806,1,paclitaxel 100 mg injection,67.0
6,39df748d-1ead-461d-5ed5-54fa622396f1,3066,0,paclitaxel 100 mg injection,61.0
7,71d99b97-f1ff-579f-a626-619ad77fda13,3262,1,paclitaxel 100 mg injection,43.0
8,86abebe8-0e13-ebe4-99e1-e91918d8f5a7,48,1,paclitaxel 100 mg injection,12.0
9,483ae6d2-0ae6-e8ec-4eed-5ce2a4a64143,1249,1,paclitaxel 100 mg injection,58.0


In [23]:
#Load dataframe with Tables method in Jupytab so that it can be exposed to Tableau
tables = jupytab.Tables()
tables['extracted_data'] = jupytab.DataFrameTable("extracted_data", standard_format, include_index=False)

In [24]:
# GET /schema
tables.render_schema()

[{"id": "extracted_data", "alias": "extracted_data", "columns": [{"id": "Id", "dataType": "string"}, {"id": "Time", "dataType": "int"}, {"id": "Status", "dataType": "int"}, {"id": "Medication", "dataType": "string"}, {"id": "Age_cancer", "dataType": "float"}]}]


In [25]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not available outside jupytab context")

Not available outside jupytab context
